In [1]:
import numpy as np

path_prefix = "./all/data/"
word_to_index = {} # map word to index
g_word_list = [] # sorted by frequence low to high
total_word_dict = {}
skip_words = set(['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'as', 'an', 'and', 'are', 'at', 'by', 'down', 'for', 'from', 'front', 'has', 'in', 'is', 'it', 'near', 'next', 'of', 'on', 'out', 'over', 'several', 'sits', 'some', 'that', 'the', 'there', 'through', 'to', 'top', 'up', 'while', 'with'])
train_des = [] #0/1/2 array of word
test_des = [] #0/1/2 array of word
train_feature_list = [] # list of feature_array by index
test_feature_list = []
all_predict_test_des = []
image_predict_by_test_description = []
train_g_category_tag = []
train_g_item_tag = []
test_g_category_tag = []
test_g_item_tag = []
MAX_INPUT = 10000
MAX_OUTPUT = 2000
# COMPONENTS = 400
# PCA_NUMBER = 2000
# ITER = 500

In [2]:
def read_file_to_string(file_name):
    f = open(file_name, "r")
    content = f.read()
    f.close()
    content = content.lower()
    content = content.replace(",", " ").replace(".", " ").replace("''", " ").replace("\"", " ")
    return content

def read_file_to_total_word_dict(file_name):
    word_list = read_file_to_string(file_name).split()
    for name in word_list:
        if name in skip_words:
            continue
        if name in total_word_dict:
            total_word_dict[name] += 1
            continue
        else:
            total_word_dict[name] = 1
            continue

def print_word_in_array(a):
    ret = ""
    for i in range(len(a)):
        if a[i] != 0:
            ret = ret + str(a[i]) + g_word_list[i] + " "
    print(ret)

In [3]:
total_word_dict = {}
for i in range(MAX_INPUT):
    read_file_to_total_word_dict(path_prefix+"descriptions_train/"+str(i)+".txt")

In [4]:
import functools
def cmp(a, b):
    a_num = abs(float(a.split("___")[0]))
    b_num = abs(float(b.split("___")[0]))
    if (a_num < b_num):
        return -1
    return 1

In [5]:
# load g_word_list
g_word_list = []
word_to_index = {}
tmp_list = []
#TODO: maybe try filter on number
for word in total_word_dict:
    if total_word_dict[word] <= 0:
        continue
    tmp_list.append(str(total_word_dict[word])+"___"+word)

tmp_list = sorted(tmp_list, key=functools.cmp_to_key(cmp))

for i in range(len(tmp_list)):
    word = tmp_list[i].split("___")[1]
    g_word_list.append(word)
    word_to_index[word] = i

In [6]:
# read to tags
train_g_category_tag = [0] * MAX_INPUT
train_g_item_tag = [0] * MAX_INPUT
test_g_category_tag = [0] * MAX_OUTPUT
test_g_item_tag = [0] * MAX_OUTPUT

#TODO: add tags in descriptions
def is_in_tag(file_number, word, train):
    if train:
        return (word in train_g_category_tag[file_number]) or (word in train_g_item_tag[file_number])
    return (word in test_g_category_tag[file_number]) or (word in test_g_item_tag[file_number])

def read_tags(file_name, train):
    lines = read_file_to_string(file_name).split("\n")
    index = int(file_name.split("/")[-1].replace(".txt", ""))
    current_category_set = set([])
    current_item_set = set([])
    
    for line in lines:
        cat = line.split(":")[0].strip()
        if cat in word_to_index:
            current_category_set.add(cat)

        item = ''
        if (cat.strip() != line.replace(":", "").strip()):
            item = line.split(":")[1].strip()
        if item in word_to_index:
            current_item_set.add(item)
    if (train):
        train_g_category_tag[index] = current_category_set
        train_g_item_tag[index] = current_item_set
    else:
        test_g_category_tag[index] = current_category_set
        test_g_item_tag[index] = current_item_set
    

for i in range(MAX_INPUT):
    read_tags(path_prefix+"tags_train/"+str(i)+".txt", True)
for i in range(MAX_OUTPUT):
    read_tags(path_prefix+"tags_test/"+str(i)+".txt", False)

In [7]:
def my_weight_func(index, count, train, file_num):
    if (count <= 0):
        return 0
    return 1

def weight_occurance_array(occurance_array, train, file_num):
    ret = [0] * len(occurance_array)
    for index in range(len(occurance_array)):
        ret[index] = my_weight_func(index, occurance_array[index], train, file_num)
    return ret

def read_des_to_list(path, file_num):
    ret = []
    train = (path.find("train") != -1)
    for i in range(file_num):
        int_array = [0] * len(g_word_list)
        temp_list = read_file_to_string(path+str(i)+".txt").split()
        for word in temp_list:
            if not word in word_to_index:
                continue
            int_array[word_to_index[word]] += 1
        ret.append(weight_occurance_array(int_array, train, i))
    return ret

train_des = read_des_to_list(path_prefix+"descriptions_train/", MAX_INPUT)
test_des = read_des_to_list(path_prefix+"descriptions_test/", MAX_OUTPUT)

In [8]:
# load feature list
def load_feature_into_list(filename):
    f = open(filename, "r")
    content = f.readlines()
    f.close()
    train = filename.find("train")
    for line in content:
        name_index = int(line.split(".jpg,")[0].split("/")[1])
        feature_list = line.split(".jpg,")[1].split(",")
        if train != -1:
            if name_index >= MAX_INPUT:
                continue
            train_feature_list[name_index] = feature_list
        else:
            if name_index >= MAX_OUTPUT:
                continue
            test_feature_list[name_index] = feature_list

train_feature_list = [0] * MAX_INPUT
test_feature_list = [0] * MAX_OUTPUT
load_feature_into_list(path_prefix+"features_train/features_resnet1000_train.csv")
load_feature_into_list(path_prefix+"features_test/features_resnet1000_test.csv")

train_feature_list = np.asarray(train_feature_list, dtype=np.float64)
test_feature_list = np.asarray(test_feature_list, dtype=np.float64)
train_des = np.asarray(train_des, dtype=np.float64)
test_des = np.asarray(test_des, dtype=np.float64)

In [9]:
from sklearn.neural_network import MLPRegressor
clf = MLPRegressor(hidden_layer_sizes=(2048,1024,512), max_iter=500, solver='sgd', verbose=True)
clf.fit(train_des, train_feature_list)

Iteration 1, loss = 2.05991197
Iteration 2, loss = 1.37333846
Iteration 3, loss = 1.15523774
Iteration 4, loss = 1.03537539
Iteration 5, loss = 0.94784513
Iteration 6, loss = 0.87158337
Iteration 7, loss = 0.80596921
Iteration 8, loss = 0.74572877
Iteration 9, loss = 0.69875319
Iteration 10, loss = 0.65252362
Iteration 11, loss = 0.61153443
Iteration 12, loss = 0.57370026
Iteration 13, loss = 0.54547666
Iteration 14, loss = 0.51714797
Iteration 15, loss = 0.48984703
Iteration 16, loss = 0.46489238
Iteration 17, loss = 0.44655504
Iteration 18, loss = 0.42371192
Iteration 19, loss = 0.40887735
Iteration 20, loss = 0.39477264
Iteration 21, loss = 0.37980894
Iteration 22, loss = 0.36342922
Iteration 23, loss = 0.35088674
Iteration 24, loss = 0.33939187
Iteration 25, loss = 0.32654824
Iteration 26, loss = 0.31637934
Iteration 27, loss = 0.30618353
Iteration 28, loss = 0.29585958
Iteration 29, loss = 0.28578220
Iteration 30, loss = 0.27773768
Iteration 31, loss = 0.26914189
Iteration 32, los

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2048, 1024, 512), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [10]:
print(train_feature_list.shape)
print(test_feature_list.shape)

(10000, 1000)
(2000, 1000)


In [11]:
print(train_des.shape)
print(test_des.shape)

(10000, 9952)
(2000, 9952)


In [12]:
print("Start 3")
image_predict_by_test_description = clf.predict(test_des)
print("Done 3")

Start 3
Done 3


In [13]:
import sklearn.metrics

# return a score
def compare_feature(target, given):
    return float(sklearn.metrics.pairwise.cosine_distances([np.array(target, dtype=float)], [np.array(given, dtype=float)]))

def get_top_20(target):
    ret = []
    for index in range(MAX_OUTPUT):
        given = test_feature_list[index]
        score = compare_feature(target, given)
        item = str(score) + "___" + str(index)
        ret.append(item)
    return sorted(ret, key=functools.cmp_to_key(cmp))[:20]

f = open("forward_nocount.csv", "w")
f.write("Descritpion_ID,Top_20_Image_IDs\n")
for index in range(MAX_OUTPUT):
    target_feature = image_predict_by_test_description[index]
    twenty_image_list = get_top_20(target_feature)
    string_to_write = "" + str(index) + ".txt,"
    for name in twenty_image_list:
        file_name = name.split("___")[1] + ".jpg"+" "
        string_to_write += file_name
    string_to_write = string_to_write.strip() + "\n"
    print(string_to_write)
    f.write(string_to_write)

f.close()

0.txt,1380.jpg 1011.jpg 1028.jpg 714.jpg 1077.jpg 1480.jpg 1969.jpg 881.jpg 95.jpg 329.jpg 540.jpg 922.jpg 46.jpg 88.jpg 1161.jpg 345.jpg 231.jpg 1992.jpg 1409.jpg 105.jpg

1.txt,57.jpg 144.jpg 355.jpg 179.jpg 1733.jpg 571.jpg 1714.jpg 1049.jpg 331.jpg 80.jpg 927.jpg 169.jpg 1172.jpg 820.jpg 1559.jpg 378.jpg 997.jpg 572.jpg 428.jpg 1989.jpg

2.txt,634.jpg 838.jpg 1292.jpg 1183.jpg 42.jpg 1724.jpg 159.jpg 1574.jpg 1471.jpg 1716.jpg 1084.jpg 117.jpg 511.jpg 904.jpg 1107.jpg 445.jpg 953.jpg 1033.jpg 1429.jpg 1208.jpg

3.txt,770.jpg 1207.jpg 1765.jpg 1187.jpg 1215.jpg 1486.jpg 1052.jpg 1620.jpg 1318.jpg 26.jpg 1727.jpg 1404.jpg 1850.jpg 1818.jpg 549.jpg 267.jpg 207.jpg 1315.jpg 236.jpg 1886.jpg

4.txt,1011.jpg 105.jpg 1296.jpg 639.jpg 1581.jpg 36.jpg 1980.jpg 1818.jpg 82.jpg 683.jpg 1836.jpg 1077.jpg 1479.jpg 969.jpg 1248.jpg 881.jpg 597.jpg 305.jpg 760.jpg 345.jpg

5.txt,1693.jpg 1292.jpg 771.jpg 1630.jpg 1335.jpg 1429.jpg 1574.jpg 51.jpg 630.jpg 249.jpg 1700.jpg 846.jpg 1991.jpg 1183.jpg

47.txt,925.jpg 1767.jpg 852.jpg 573.jpg 986.jpg 941.jpg 1632.jpg 905.jpg 709.jpg 817.jpg 72.jpg 1661.jpg 1930.jpg 148.jpg 1515.jpg 731.jpg 1430.jpg 441.jpg 471.jpg 884.jpg

48.txt,953.jpg 445.jpg 600.jpg 1183.jpg 904.jpg 879.jpg 426.jpg 1533.jpg 279.jpg 634.jpg 333.jpg 1144.jpg 873.jpg 539.jpg 1118.jpg 523.jpg 998.jpg 1089.jpg 1713.jpg 1292.jpg

49.txt,595.jpg 1202.jpg 603.jpg 1924.jpg 57.jpg 1136.jpg 130.jpg 1351.jpg 877.jpg 58.jpg 867.jpg 1003.jpg 1743.jpg 1201.jpg 299.jpg 435.jpg 592.jpg 154.jpg 736.jpg 1110.jpg

50.txt,1290.jpg 1534.jpg 1302.jpg 1190.jpg 183.jpg 836.jpg 560.jpg 1223.jpg 91.jpg 1469.jpg 1703.jpg 1362.jpg 1432.jpg 1711.jpg 932.jpg 1832.jpg 384.jpg 1085.jpg 1694.jpg 689.jpg

51.txt,1053.jpg 305.jpg 514.jpg 1980.jpg 289.jpg 829.jpg 868.jpg 1630.jpg 770.jpg 309.jpg 728.jpg 951.jpg 413.jpg 1700.jpg 105.jpg 1903.jpg 1479.jpg 549.jpg 1088.jpg 1400.jpg

52.txt,1746.jpg 880.jpg 1798.jpg 1043.jpg 1213.jpg 390.jpg 617.jpg 138.jpg 1926.jpg 1715.jpg 498.jpg 151.jpg 273.jpg 544.j

94.txt,1408.jpg 1550.jpg 331.jpg 538.jpg 225.jpg 1199.jpg 1029.jpg 1068.jpg 80.jpg 835.jpg 885.jpg 1143.jpg 1733.jpg 1116.jpg 1173.jpg 1806.jpg 529.jpg 1807.jpg 818.jpg 23.jpg

95.txt,846.jpg 760.jpg 1533.jpg 82.jpg 998.jpg 1296.jpg 1913.jpg 1355.jpg 1693.jpg 492.jpg 105.jpg 1292.jpg 1630.jpg 1118.jpg 413.jpg 1700.jpg 1254.jpg 1479.jpg 1429.jpg 124.jpg

96.txt,696.jpg 904.jpg 883.jpg 634.jpg 1183.jpg 159.jpg 322.jpg 511.jpg 579.jpg 979.jpg 445.jpg 1896.jpg 1840.jpg 1471.jpg 426.jpg 333.jpg 953.jpg 838.jpg 539.jpg 476.jpg

97.txt,1664.jpg 953.jpg 445.jpg 117.jpg 1487.jpg 561.jpg 690.jpg 979.jpg 634.jpg 1716.jpg 1040.jpg 904.jpg 180.jpg 1144.jpg 765.jpg 328.jpg 873.jpg 1853.jpg 600.jpg 1033.jpg

98.txt,899.jpg 1265.jpg 458.jpg 262.jpg 414.jpg 1913.jpg 1151.jpg 1789.jpg 429.jpg 1886.jpg 623.jpg 712.jpg 1191.jpg 1465.jpg 1458.jpg 1333.jpg 1818.jpg 940.jpg 1118.jpg 413.jpg

99.txt,1178.jpg 1825.jpg 1585.jpg 1683.jpg 612.jpg 830.jpg 1267.jpg 1323.jpg 1858.jpg 966.jpg 1670.jpg 1576.jpg 1374.j

141.txt,636.jpg 1189.jpg 750.jpg 1207.jpg 1301.jpg 1215.jpg 1187.jpg 1465.jpg 267.jpg 824.jpg 1778.jpg 275.jpg 1831.jpg 747.jpg 573.jpg 960.jpg 207.jpg 1552.jpg 1127.jpg 743.jpg

142.txt,890.jpg 1160.jpg 1549.jpg 301.jpg 1332.jpg 1221.jpg 622.jpg 548.jpg 27.jpg 713.jpg 1182.jpg 58.jpg 380.jpg 1531.jpg 297.jpg 34.jpg 1608.jpg 1271.jpg 596.jpg 462.jpg

143.txt,953.jpg 445.jpg 634.jpg 117.jpg 1716.jpg 1183.jpg 1144.jpg 1222.jpg 1713.jpg 322.jpg 372.jpg 115.jpg 1499.jpg 279.jpg 1664.jpg 1033.jpg 769.jpg 879.jpg 1471.jpg 505.jpg

144.txt,153.jpg 1077.jpg 881.jpg 1382.jpg 305.jpg 683.jpg 1500.jpg 231.jpg 1581.jpg 105.jpg 669.jpg 36.jpg 749.jpg 46.jpg 717.jpg 588.jpg 1404.jpg 616.jpg 1553.jpg 1850.jpg

145.txt,1246.jpg 1592.jpg 1548.jpg 1983.jpg 858.jpg 1519.jpg 1800.jpg 812.jpg 100.jpg 1963.jpg 1080.jpg 1490.jpg 324.jpg 86.jpg 524.jpg 974.jpg 1245.jpg 1652.jpg 123.jpg 1897.jpg

146.txt,561.jpg 1987.jpg 1040.jpg 338.jpg 953.jpg 998.jpg 760.jpg 244.jpg 904.jpg 184.jpg 600.jpg 1648.jpg 1107.jpg

188.txt,1946.jpg 300.jpg 1467.jpg 710.jpg 403.jpg 1104.jpg 679.jpg 1604.jpg 404.jpg 581.jpg 327.jpg 664.jpg 362.jpg 241.jpg 1829.jpg 551.jpg 1947.jpg 1747.jpg 1425.jpg 283.jpg

189.txt,573.jpg 207.jpg 474.jpg 1215.jpg 884.jpg 1632.jpg 1430.jpg 1189.jpg 618.jpg 1778.jpg 1128.jpg 1765.jpg 731.jpg 1552.jpg 1767.jpg 412.jpg 1595.jpg 72.jpg 1620.jpg 960.jpg

190.txt,1640.jpg 1744.jpg 1150.jpg 1579.jpg 369.jpg 1685.jpg 1140.jpg 686.jpg 1260.jpg 340.jpg 1038.jpg 494.jpg 1846.jpg 407.jpg 239.jpg 1859.jpg 53.jpg 1587.jpg 1953.jpg 1438.jpg

191.txt,1510.jpg 1087.jpg 132.jpg 959.jpg 112.jpg 619.jpg 1958.jpg 312.jpg 1384.jpg 444.jpg 1181.jpg 922.jpg 1273.jpg 1298.jpg 1161.jpg 356.jpg 1969.jpg 985.jpg 1198.jpg 40.jpg

192.txt,179.jpg 331.jpg 200.jpg 571.jpg 428.jpg 927.jpg 1049.jpg 885.jpg 144.jpg 355.jpg 1199.jpg 1733.jpg 80.jpg 1824.jpg 1408.jpg 160.jpg 1172.jpg 1806.jpg 1029.jpg 1742.jpg

193.txt,577.jpg 932.jpg 1363.jpg 975.jpg 10.jpg 1123.jpg 1949.jpg 591.jpg 226.jpg 560.jpg 384.jpg 1832.jpg 1

...

1813.txt,1275.jpg 1151.jpg 1458.jpg 262.jpg 1289.jpg 996.jpg 1472.jpg 655.jpg 535.jpg 1944.jpg 897.jpg 1660.jpg 1312.jpg 156.jpg 1540.jpg 1311.jpg 458.jpg 910.jpg 899.jpg 1758.jpg

1814.txt,573.jpg 1301.jpg 1189.jpg 1571.jpg 1552.jpg 1128.jpg 1418.jpg 1767.jpg 1403.jpg 743.jpg 145.jpg 207.jpg 275.jpg 933.jpg 824.jpg 1620.jpg 884.jpg 235.jpg 817.jpg 1358.jpg

1815.txt,1463.jpg 1450.jpg 779.jpg 661.jpg 650.jpg 677.jpg 1819.jpg 1279.jpg 59.jpg 764.jpg 1528.jpg 1125.jpg 930.jpg 239.jpg 320.jpg 944.jpg 577.jpg 432.jpg 549.jpg 1953.jpg

1816.txt,183.jpg 1190.jpg 1290.jpg 1708.jpg 1078.jpg 1832.jpg 1711.jpg 1432.jpg 1469.jpg 1302.jpg 932.jpg 560.jpg 1534.jpg 1022.jpg 836.jpg 1875.jpg 1703.jpg 384.jpg 91.jpg 1123.jpg

1817.txt,982.jpg 1644.jpg 848.jpg 420.jpg 844.jpg 1179.jpg 459.jpg 1336.jpg 1690.jpg 1389.jpg 1013.jpg 1365.jpg 343.jpg 1525.jpg 521.jpg 1896.jpg 891.jpg 1010.jpg 1611.jpg 377.jpg

1818.txt,603.jpg 867.jpg 1392.jpg 443.jpg 697.jpg 1793.jpg 1003.jpg 1907.jpg 1851.jpg 1194.jpg 130.

1859.txt,1946.jpg 710.jpg 300.jpg 1467.jpg 1104.jpg 551.jpg 362.jpg 404.jpg 1604.jpg 283.jpg 403.jpg 241.jpg 679.jpg 1947.jpg 327.jpg 1829.jpg 581.jpg 195.jpg 873.jpg 1666.jpg

1860.txt,770.jpg 1318.jpg 413.jpg 1400.jpg 829.jpg 1187.jpg 1620.jpg 1850.jpg 1423.jpg 309.jpg 82.jpg 899.jpg 1088.jpg 1791.jpg 77.jpg 1162.jpg 1818.jpg 1980.jpg 760.jpg 1913.jpg

1861.txt,327.jpg 1829.jpg 1104.jpg 195.jpg 710.jpg 1947.jpg 1467.jpg 277.jpg 1766.jpg 241.jpg 1946.jpg 1604.jpg 1244.jpg 551.jpg 1666.jpg 558.jpg 1583.jpg 1390.jpg 1993.jpg 283.jpg

1862.txt,108.jpg 1072.jpg 1113.jpg 102.jpg 232.jpg 1914.jpg 1314.jpg 793.jpg 1684.jpg 1090.jpg 313.jpg 1245.jpg 935.jpg 1780.jpg 1375.jpg 40.jpg 84.jpg 1626.jpg 1268.jpg 1912.jpg

1863.txt,1202.jpg 820.jpg 1039.jpg 1230.jpg 1351.jpg 746.jpg 58.jpg 1835.jpg 9.jpg 76.jpg 548.jpg 595.jpg 201.jpg 357.jpg 1735.jpg 1639.jpg 525.jpg 1257.jpg 1782.jpg 1120.jpg

1864.txt,1137.jpg 1504.jpg 1502.jpg 1342.jpg 799.jpg 617.jpg 1171.jpg 295.jpg 926.jpg 1236.jpg 75.jpg 605

1905.txt,524.jpg 102.jpg 1245.jpg 387.jpg 1626.jpg 1090.jpg 1113.jpg 1548.jpg 1912.jpg 1519.jpg 935.jpg 40.jpg 1229.jpg 1780.jpg 858.jpg 1592.jpg 1478.jpg 1914.jpg 1256.jpg 1873.jpg

1906.txt,760.jpg 361.jpg 1355.jpg 1700.jpg 940.jpg 413.jpg 1191.jpg 969.jpg 1630.jpg 770.jpg 429.jpg 614.jpg 998.jpg 1423.jpg 512.jpg 514.jpg 82.jpg 899.jpg 1479.jpg 1296.jpg

1907.txt,1553.jpg 749.jpg 1720.jpg 981.jpg 1919.jpg 725.jpg 1500.jpg 1383.jpg 1883.jpg 153.jpg 487.jpg 476.jpg 290.jpg 1382.jpg 1023.jpg 106.jpg 739.jpg 717.jpg 1788.jpg 157.jpg

1908.txt,1410.jpg 1980.jpg 92.jpg 1836.jpg 305.jpg 698.jpg 93.jpg 678.jpg 1053.jpg 1992.jpg 1286.jpg 647.jpg 541.jpg 105.jpg 723.jpg 1627.jpg 222.jpg 669.jpg 1231.jpg 46.jpg

1909.txt,499.jpg 1356.jpg 780.jpg 1019.jpg 1422.jpg 1994.jpg 578.jpg 1058.jpg 1964.jpg 324.jpg 1873.jpg 827.jpg 1349.jpg 733.jpg 1710.jpg 707.jpg 291.jpg 278.jpg 1554.jpg 396.jpg

1910.txt,728.jpg 155.jpg 951.jpg 914.jpg 1903.jpg 1630.jpg 172.jpg 1889.jpg 514.jpg 522.jpg 249.jpg 1791.jp

1951.txt,1012.jpg 1235.jpg 268.jpg 64.jpg 439.jpg 1621.jpg 777.jpg 370.jpg 1376.jpg 128.jpg 853.jpg 673.jpg 87.jpg 1364.jpg 1813.jpg 708.jpg 440.jpg 972.jpg 1966.jpg 214.jpg

1952.txt,590.jpg 144.jpg 820.jpg 61.jpg 572.jpg 179.jpg 571.jpg 611.jpg 1049.jpg 238.jpg 927.jpg 774.jpg 997.jpg 888.jpg 1006.jpg 57.jpg 169.jpg 1688.jpg 1347.jpg 1988.jpg

1953.txt,972.jpg 439.jpg 708.jpg 1633.jpg 777.jpg 1984.jpg 289.jpg 853.jpg 1966.jpg 1657.jpg 1882.jpg 1235.jpg 87.jpg 268.jpg 1339.jpg 370.jpg 1012.jpg 128.jpg 1154.jpg 1558.jpg

1954.txt,1807.jpg 859.jpg 1068.jpg 12.jpg 121.jpg 538.jpg 118.jpg 1937.jpg 1726.jpg 885.jpg 1196.jpg 1110.jpg 1408.jpg 877.jpg 331.jpg 571.jpg 1989.jpg 1184.jpg 1120.jpg 1806.jpg

1955.txt,988.jpg 1915.jpg 1356.jpg 548.jpg 2.jpg 1710.jpg 1354.jpg 499.jpg 1050.jpg 1182.jpg 44.jpg 733.jpg 1549.jpg 1274.jpg 363.jpg 780.jpg 706.jpg 556.jpg 1490.jpg 694.jpg

1956.txt,1273.jpg 1561.jpg 132.jpg 112.jpg 1384.jpg 525.jpg 1466.jpg 959.jpg 1161.jpg 1087.jpg 2.jpg 922.jpg 912.jpg 

1997.txt,1564.jpg 1958.jpg 1266.jpg 178.jpg 104.jpg 58.jpg 27.jpg 782.jpg 657.jpg 1198.jpg 596.jpg 293.jpg 462.jpg 1055.jpg 11.jpg 1868.jpg 1419.jpg 855.jpg 34.jpg 1531.jpg

1998.txt,112.jpg 959.jpg 1273.jpg 44.jpg 1510.jpg 484.jpg 1738.jpg 34.jpg 1384.jpg 1087.jpg 1969.jpg 1466.jpg 956.jpg 1161.jpg 21.jpg 1294.jpg 1131.jpg 1535.jpg 912.jpg 1958.jpg

1999.txt,1798.jpg 68.jpg 1213.jpg 138.jpg 654.jpg 772.jpg 1504.jpg 285.jpg 1502.jpg 153.jpg 389.jpg 1589.jpg 390.jpg 373.jpg 468.jpg 898.jpg 604.jpg 1856.jpg 530.jpg 880.jpg

